# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** 
decoder: runs resnet50, and outputs a connection layer with  dimention equels to the first input for the encoder (so, output the embedding size as parameter), no use of last softmax activation because we dont use classification so it removed.
encoder: we start from the latent space vector that was outputed from the decoder as input for t=0. 
then we use the captions for the first t, and "feed" them to the first word embedding layer, we then concat them together to feed the lstm cell (we only have one layer of lstm). befor the first step we iniate the hidden first step only and set it to zero, we than use that layer for the first lstm step, who receives all thr cations one by one which followed by a linear layer to output the best fit word from the lstm as best fit for the next word (which is received from the training of course). 
batch = 200 is a good size when you have a strong gpu, vocab_threshold = 6 good size vocabulary - not to big, 
embed_size = 312 as the input, bigger maybe tends to overfit - after a few tries I got to this, 
hidden_size = 250, ones again, after a few tries that was good enough, num_epochs = 3 as  you suggested. 



### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** 
I left it how it was. the transforms are good and fit the needs of the restnet architecture. with the addition of hflip, we get  some more variety to add complexity and  reduce the chance of overfitting.
### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** 
we are using transferred learning for the decoder, which is able to detect patterns in a good way. allowing to train only the  embedding weights for the captions (lstm), we ensure we can focus on finding the right sequence of words, without having to train the heavy part of the cnn for days or even destroying the ability to generelize that the CNN have.

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:**
for the loss,  CrossEntropyLoss is the obvies, but the paper recommend using another loss called "doubly stochastic regularization". this makes the model pay more attention to all the time steps. i might add this in the next stage.

I chose Adam() as an optimizer with an initial learning rate of 0.001 - adam is updating the learning rate as we progress.


In [3]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math
import nltk
nltk.download()
# 'punkt'

## TODO #1: Select appropriate values for the Python variables below.
# batch_size = 64          # first time
batch_size = 200          #lets enjoy your gpu
vocab_threshold = 6        #big enough size of vocab
vocab_from_file = True    # if True, load existing vocab file
# embed_size = 512           # dimensionality of image and word embeddings - as the paper suggested
embed_size = 312  #so, at second try, after not seeing the improvment  iwould like to see, 
# i decided to decrease the number of embed size  maybe it was to large to get to generelize
hidden_size = 250          # number of features in hidden state of the RNN decoder - decreased the hidden size
num_epochs = 3             # number of training epochs - as you suggested
save_every = 1             # determines frequency of saving model weights
print_every = 150          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params = params, lr = 0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> punkt
      Unzipping tokenizers/punkt.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q
Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...




  0%|          | 0/414113 [00:00<?, ?it/s]

  0%|          | 343/414113 [00:00<02:00, 3428.16it/s]

  0%|          | 762/414113 [00:00<01:54, 3624.40it/s]

  0%|          | 1203/414113 [00:00<01:47, 3828.62it/s]

  0%|          | 1640/414113 [00:00<01:43, 3976.33it/s]

  1%|          | 2080/414113 [00:00<01:40, 4092.25it/s]

  1%|          | 2519/414113 [00:00<01:38, 4176.17it/s]

  1%|          | 2959/414113 [00:00<01:36, 4239.84it/s]

  1%|          | 3383/414113 [00:00<01:36, 4239.49it/s]

  1%|          | 3818/414113 [00:00<01:36, 4270.33it/s]

  1%|          | 4257/414113 [00:01<01:35, 4303.39it/s]

  1%|          | 4709/414113 [00:01<01:33, 4365.49it/s]

  1%|          | 5159/414113 [00:01<01:32, 4404.35it/s]

  1%|▏         | 5601/414113 [00:01<01:32, 4408.44it/s]

  1%|▏         | 6040/414113 [00:01<01:37, 4200.16it/s]

  2%|▏         | 6468/414113 [00:01<01:36, 4222.52it/s]

  2%|▏         | 6908/414113 [00:01<01:35, 4272.08it/s]

  2%|▏         | 7337/414113 [00:01<01:35, 4

 14%|█▎        | 56706/414113 [00:13<02:24, 2476.39it/s]

 14%|█▍        | 57156/414113 [00:13<02:04, 2861.78it/s]

 14%|█▍        | 57612/414113 [00:13<01:50, 3221.26it/s]

 14%|█▍        | 58046/414113 [00:13<01:42, 3489.93it/s]

 14%|█▍        | 58486/414113 [00:13<01:35, 3718.32it/s]

 14%|█▍        | 58935/414113 [00:13<01:30, 3918.42it/s]

 14%|█▍        | 59390/414113 [00:13<01:26, 4088.30it/s]

 14%|█▍        | 59838/414113 [00:13<01:24, 4196.45it/s]

 15%|█▍        | 60292/414113 [00:14<01:22, 4291.77it/s]

 15%|█▍        | 60751/414113 [00:14<01:20, 4376.67it/s]

 15%|█▍        | 61200/414113 [00:14<01:20, 4366.45it/s]

 15%|█▍        | 61648/414113 [00:14<01:20, 4397.64it/s]

 15%|█▍        | 62093/414113 [00:14<01:20, 4398.08it/s]

 15%|█▌        | 62543/414113 [00:14<01:19, 4427.55it/s]

 15%|█▌        | 62997/414113 [00:14<01:18, 4459.11it/s]

 15%|█▌        | 63445/414113 [00:14<01:18, 4465.25it/s]

 15%|█▌        | 63895/414113 [00:14<01:18, 4473.48it/s]

 16%|█▌       

 27%|██▋       | 113623/414113 [00:26<01:07, 4427.81it/s]

 28%|██▊       | 114085/414113 [00:26<01:06, 4481.87it/s]

 28%|██▊       | 114534/414113 [00:26<01:07, 4423.24it/s]

 28%|██▊       | 114986/414113 [00:26<01:07, 4449.77it/s]

 28%|██▊       | 115443/414113 [00:26<01:06, 4484.29it/s]

 28%|██▊       | 115892/414113 [00:26<01:06, 4463.70it/s]

 28%|██▊       | 116339/414113 [00:26<01:06, 4458.53it/s]

 28%|██▊       | 116786/414113 [00:26<01:07, 4421.42it/s]

 28%|██▊       | 117229/414113 [00:26<01:07, 4415.01it/s]

 28%|██▊       | 117689/414113 [00:26<01:06, 4467.67it/s]

 29%|██▊       | 118137/414113 [00:27<01:06, 4457.06it/s]

 29%|██▊       | 118583/414113 [00:27<01:06, 4449.57it/s]

 29%|██▊       | 119029/414113 [00:27<01:06, 4442.00it/s]

 29%|██▉       | 119474/414113 [00:27<01:06, 4425.98it/s]

 29%|██▉       | 119933/414113 [00:27<01:05, 4472.20it/s]

 29%|██▉       | 120388/414113 [00:27<01:05, 4495.05it/s]

 29%|██▉       | 120838/414113 [00:27<01:05, 4483.58it/s

 41%|████      | 170335/414113 [00:38<00:56, 4306.89it/s]

 41%|████      | 170769/414113 [00:38<00:56, 4316.10it/s]

 41%|████▏     | 171218/414113 [00:39<00:55, 4365.98it/s]

 41%|████▏     | 171659/414113 [00:39<00:55, 4377.88it/s]

 42%|████▏     | 172104/414113 [00:39<00:55, 4397.36it/s]

 42%|████▏     | 172553/414113 [00:39<00:54, 4422.52it/s]

 42%|████▏     | 172996/414113 [00:39<00:55, 4350.72it/s]

 42%|████▏     | 173436/414113 [00:39<00:55, 4365.38it/s]

 42%|████▏     | 173873/414113 [00:39<01:44, 2307.05it/s]

 42%|████▏     | 174302/414113 [00:40<01:29, 2677.82it/s]

 42%|████▏     | 174750/414113 [00:40<01:18, 3044.66it/s]

 42%|████▏     | 175179/414113 [00:40<01:11, 3333.91it/s]

 42%|████▏     | 175608/414113 [00:40<01:06, 3570.99it/s]

 43%|████▎     | 176018/414113 [00:40<01:04, 3706.10it/s]

 43%|████▎     | 176460/414113 [00:40<01:01, 3893.67it/s]

 43%|████▎     | 176921/414113 [00:40<00:58, 4081.10it/s]

 43%|████▎     | 177352/414113 [00:40<00:57, 4143.18it/s

 55%|█████▍    | 226946/414113 [00:51<00:40, 4602.50it/s]

 55%|█████▍    | 227407/414113 [00:51<00:41, 4497.34it/s]

 55%|█████▌    | 227870/414113 [00:52<00:41, 4536.24it/s]

 55%|█████▌    | 228325/414113 [00:52<00:41, 4523.52it/s]

 55%|█████▌    | 228778/414113 [00:52<00:41, 4498.39it/s]

 55%|█████▌    | 229229/414113 [00:52<00:41, 4439.02it/s]

 55%|█████▌    | 229674/414113 [00:52<00:42, 4338.03it/s]

 56%|█████▌    | 230117/414113 [00:52<00:42, 4363.03it/s]

 56%|█████▌    | 230554/414113 [00:52<00:42, 4327.22it/s]

 56%|█████▌    | 230992/414113 [00:52<00:42, 4341.53it/s]

 56%|█████▌    | 231437/414113 [00:52<00:41, 4373.29it/s]

 56%|█████▌    | 231892/414113 [00:53<00:41, 4423.90it/s]

 56%|█████▌    | 232335/414113 [00:53<00:41, 4375.73it/s]

 56%|█████▌    | 232773/414113 [00:53<00:41, 4372.88it/s]

 56%|█████▋    | 233229/414113 [00:53<00:40, 4425.53it/s]

 56%|█████▋    | 233689/414113 [00:53<00:40, 4474.34it/s]

 57%|█████▋    | 234137/414113 [00:53<00:40, 4459.93it/s

 68%|██████▊   | 283551/414113 [01:04<00:28, 4526.21it/s]

 69%|██████▊   | 284004/414113 [01:04<00:29, 4435.52it/s]

 69%|██████▊   | 284456/414113 [01:04<00:29, 4460.45it/s]

 69%|██████▉   | 284912/414113 [01:04<00:28, 4487.97it/s]

 69%|██████▉   | 285362/414113 [01:05<00:28, 4449.17it/s]

 69%|██████▉   | 285813/414113 [01:05<00:28, 4466.22it/s]

 69%|██████▉   | 286260/414113 [01:05<00:28, 4457.15it/s]

 69%|██████▉   | 286710/414113 [01:05<00:28, 4469.05it/s]

 69%|██████▉   | 287158/414113 [01:05<00:28, 4454.41it/s]

 69%|██████▉   | 287609/414113 [01:05<00:28, 4468.73it/s]

 70%|██████▉   | 288068/414113 [01:05<00:27, 4504.36it/s]

 70%|██████▉   | 288519/414113 [01:05<00:27, 4490.80it/s]

 70%|██████▉   | 288969/414113 [01:05<00:27, 4474.08it/s]

 70%|██████▉   | 289417/414113 [01:05<00:27, 4473.75it/s]

 70%|███████   | 289888/414113 [01:06<00:27, 4539.37it/s]

 70%|███████   | 290343/414113 [01:06<00:27, 4477.39it/s]

 70%|███████   | 290795/414113 [01:06<00:27, 4487.92it/s

 82%|████████▏ | 340122/414113 [01:17<00:17, 4338.12it/s]

 82%|████████▏ | 340558/414113 [01:17<00:16, 4344.52it/s]

 82%|████████▏ | 340998/414113 [01:17<00:16, 4360.33it/s]

 82%|████████▏ | 341435/414113 [01:18<00:16, 4353.04it/s]

 83%|████████▎ | 341882/414113 [01:18<00:16, 4385.65it/s]

 83%|████████▎ | 342327/414113 [01:18<00:16, 4403.87it/s]

 83%|████████▎ | 342768/414113 [01:18<00:16, 4380.98it/s]

 83%|████████▎ | 343212/414113 [01:18<00:16, 4398.05it/s]

 83%|████████▎ | 343656/414113 [01:18<00:15, 4410.41it/s]

 83%|████████▎ | 344109/414113 [01:18<00:15, 4443.63it/s]

 83%|████████▎ | 344560/414113 [01:18<00:15, 4461.16it/s]

 83%|████████▎ | 345011/414113 [01:18<00:15, 4475.69it/s]

 83%|████████▎ | 345459/414113 [01:18<00:15, 4475.38it/s]

 84%|████████▎ | 345907/414113 [01:19<00:15, 4466.86it/s]

 84%|████████▎ | 346364/414113 [01:19<00:15, 4496.64it/s]

 84%|████████▎ | 346814/414113 [01:19<00:14, 4489.14it/s]

 84%|████████▍ | 347264/414113 [01:19<00:14, 4491.36it/s

 96%|█████████▌| 396482/414113 [01:30<00:03, 4439.76it/s]

 96%|█████████▌| 396927/414113 [01:30<00:03, 4435.09it/s]

 96%|█████████▌| 397371/414113 [01:30<00:03, 4341.79it/s]

 96%|█████████▌| 397811/414113 [01:30<00:03, 4359.00it/s]

 96%|█████████▌| 398248/414113 [01:30<00:03, 4352.49it/s]

 96%|█████████▋| 398698/414113 [01:31<00:03, 4393.00it/s]

 96%|█████████▋| 399138/414113 [01:31<00:03, 4368.91it/s]

 96%|█████████▋| 399576/414113 [01:31<00:03, 4371.73it/s]

 97%|█████████▋| 400023/414113 [01:31<00:03, 4400.54it/s]

 97%|█████████▋| 400464/414113 [01:31<00:03, 4335.49it/s]

 97%|█████████▋| 400898/414113 [01:31<00:03, 4332.49it/s]

 97%|█████████▋| 401332/414113 [01:31<00:02, 4326.75it/s]

 97%|█████████▋| 401769/414113 [01:31<00:02, 4337.30it/s]

 97%|█████████▋| 402217/414113 [01:31<00:02, 4378.19it/s]

 97%|█████████▋| 402659/414113 [01:31<00:02, 4389.57it/s]

 97%|█████████▋| 403099/414113 [01:32<00:02, 4347.85it/s]

 97%|█████████▋| 403536/414113 [01:32<00:02, 4354.40it/s

Done (t=0.90s)
creating index...
index created!
Obtaining caption lengths...


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [ ]:
# first attemp

In [ ]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [100/6471], Loss: 4.1382, Perplexity: 62.68868
Epoch [1/3], Step [200/6471], Loss: 3.8981, Perplexity: 49.3076
Epoch [1/3], Step [300/6471], Loss: 3.7886, Perplexity: 44.1958
Epoch [1/3], Step [400/6471], Loss: 3.7034, Perplexity: 40.5859
Epoch [1/3], Step [500/6471], Loss: 3.3545, Perplexity: 28.6310
Epoch [1/3], Step [600/6471], Loss: 3.4793, Perplexity: 32.4380
Epoch [1/3], Step [700/6471], Loss: 3.3007, Perplexity: 27.1310
Epoch [1/3], Step [800/6471], Loss: 3.7158, Perplexity: 41.0905
Epoch [1/3], Step [900/6471], Loss: 3.2597, Perplexity: 26.0414
Epoch [1/3], Step [1000/6471], Loss: 2.9526, Perplexity: 19.1554
Epoch [1/3], Step [1100/6471], Loss: 3.3603, Perplexity: 28.79773
Epoch [1/3], Step [1200/6471], Loss: 3.2459, Perplexity: 25.6845
Epoch [1/3], Step [1300/6471], Loss: 2.9287, Perplexity: 18.7028
Epoch [1/3], Step [1400/6471], Loss: 3.7627, Perplexity: 43.0666
Epoch [1/3], Step [1500/6471], Loss: 2.9206, Perplexity: 18.5532
Epoch [1/3], Step [1600/6471], L

In [2]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [100/6471], Loss: 4.1316, Perplexity: 62.27977
Epoch [1/3], Step [200/6471], Loss: 4.0426, Perplexity: 56.97590
Epoch [1/3], Step [300/6471], Loss: 3.9364, Perplexity: 51.2344
Epoch [1/3], Step [400/6471], Loss: 3.5770, Perplexity: 35.7677
Epoch [1/3], Step [500/6471], Loss: 3.5420, Perplexity: 34.5368
Epoch [1/3], Step [600/6471], Loss: 3.2777, Perplexity: 26.5138
Epoch [1/3], Step [700/6471], Loss: 3.2957, Perplexity: 26.9972
Epoch [1/3], Step [800/6471], Loss: 3.4295, Perplexity: 30.8624
Epoch [1/3], Step [900/6471], Loss: 3.0486, Perplexity: 21.0858
Epoch [1/3], Step [1000/6471], Loss: 3.4956, Perplexity: 32.9691
Epoch [1/3], Step [1100/6471], Loss: 3.3028, Perplexity: 27.18787
Epoch [1/3], Step [1200/6471], Loss: 3.1524, Perplexity: 23.3926
Epoch [1/3], Step [1300/6471], Loss: 3.2583, Perplexity: 26.0048
Epoch [1/3], Step [1400/6471], Loss: 3.1377, Perplexity: 23.0510
Epoch [1/3], Step [1500/6471], Loss: 3.5182, Perplexity: 33.7238
Epoch [1/3], Step [1600/6471], 

In [4]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [150/2071], Loss: 3.5488, Perplexity: 34.77138
Epoch [1/3], Step [300/2071], Loss: 3.2509, Perplexity: 25.81245
Epoch [1/3], Step [450/2071], Loss: 2.9892, Perplexity: 19.87020
Epoch [1/3], Step [600/2071], Loss: 2.9478, Perplexity: 19.0642
Epoch [1/3], Step [750/2071], Loss: 2.8102, Perplexity: 16.6125
Epoch [1/3], Step [900/2071], Loss: 2.6452, Perplexity: 14.0868
Epoch [1/3], Step [1050/2071], Loss: 3.9493, Perplexity: 51.9003
Epoch [1/3], Step [1200/2071], Loss: 2.5503, Perplexity: 12.8112
Epoch [1/3], Step [1350/2071], Loss: 2.4929, Perplexity: 12.0965
Epoch [1/3], Step [1500/2071], Loss: 2.3700, Perplexity: 10.6969
Epoch [1/3], Step [1650/2071], Loss: 2.4927, Perplexity: 12.0945
Epoch [1/3], Step [1800/2071], Loss: 2.4492, Perplexity: 11.5796
Epoch [1/3], Step [1950/2071], Loss: 2.4084, Perplexity: 11.1163
Epoch [2/3], Step [150/2071], Loss: 2.7692, Perplexity: 15.94525
Epoch [2/3], Step [300/2071], Loss: 2.3853, Perplexity: 10.8625
Epoch [2/3], Step [450/2071],

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.